# HW 5 张宇扬 2301212424

* HW1 Extract the first page of all pdfs in the "mypdfs " folder and merge them into one pdf file named "new.pdf"

In [2]:
from pathlib import Path # get pdf names in the folder
 
path = Path(r"python basic/Week 6/mypdfs") # Specify the path to the folder containing PDF files
pdf_files = [p for p in path.glob("*.pdf")] # Use a list comprehension to get all files in the folder with a ".pdf" extension
print(pdf_files) # Print the list of PDF file names

[WindowsPath('python basic/Week 6/mypdfs/1905.05406.pdf'), WindowsPath('python basic/Week 6/mypdfs/1908.02426.pdf'), WindowsPath('python basic/Week 6/mypdfs/2103.07579.pdf'), WindowsPath('python basic/Week 6/mypdfs/2106.13112.pdf'), WindowsPath('python basic/Week 6/mypdfs/2109.05673.pdf')]


In [5]:
from PyPDF4 import PdfFileReader, PdfFileMerger, PdfFileWriter #merge them into one list
newfilelist = []
for file in pdf_files: # Iterate over the previously obtained list of PDF files
    with open(file, "rb") as pdf_file: # Open the current PDF file in binary read mode
        pdf = PdfFileReader(pdf_file) # Create a PdfFileReader object to read the PDF file
        page = pdf.getPage(0) # Get the first page of the PDF file
    
        writer = PdfFileWriter() # Create a PdfFileWriter object to write the PDF file
        writer.addPage(page) # Add the retrieved page to the new PDF file
    
    # Construct a new file name by removing the original file extension (.pdf) and adding '_new.pdf' at the end
        newfile = file.name[:-4]+'_new.pdf'
        newfilelist.append(newfile)
        with open(newfile, "wb") as output:
             writer.write(output)
print(newfilelist)

['1905.05406_new.pdf', '1908.02426_new.pdf', '2103.07579_new.pdf', '2106.13112_new.pdf', '2109.05673_new.pdf']


In [6]:
merger = PdfFileMerger()

for file_name in newfilelist:
    merger.append(file_name)
merger.write("new.pdf")

* HW2 Take the abstracts of the first 20 papers in the given URL, translate them into Chinese, and 
write them into the file named 'my_abstract.txt’.


URL = https://openaccess.thecvf.com/ICCV2021?day=2021-10-12


In [95]:
def getUrl(url):
    response = requests.get(url)
    pdfUrl = []
    count = 0
    bs = BeautifulSoup(response.text,"html.parser")  # Parse the HTML content of the response using BeautifulSoup
    pdfs = bs.find_all(href=re.compile('/content/.*html')) # Find all anchor tags with href attribute matching the specified pattern
    names = []
    for tag in pdfs:
        if count < 20:  # Limit the count to 20
            pdfUrl.append("https://openaccess.thecvf.com"+tag.get("href"))
            count = count+1
            # Extract the name from the href attribute and modify it before appending to the names list
            names.append(re.split('[./]', tag.get("href"))[-2][:-16].replace("_"," ")) 
        else: break
    return names,pdfUrl


def getAbstract(paperUrls):
    abstract = []
    for url in paperUrls:
        response = requests.get(url)
        bs = BeautifulSoup(response.text,"html.parser")
        abstract.append(bs.find(id="abstract").text.strip()) #get the content of id "abstract"
    return abstract

def translate_api(input_text):
    # Set your own appid/appkey.
    appid = '20231227001923044'
    appkey = 'mTCuoFbvi0KZtJozg_MQ'
    def make_md5(s, encoding='utf-8'):
        return md5(s.encode(encoding)).hexdigest()
    # For list of language codes, please refer to `https://api.fanyi.baidu.com/doc/21`
    from_lang = 'en'
    to_lang = 'zh'
    endpoint = 'http://api.fanyi.baidu.com'
    path = '/api/trans/vip/translate'
    url = endpoint + path
    
    query = input_text
    # Generate salt and sign
    salt = random.randint(32768, 65536)
    sign = make_md5(appid + query + str(salt) + appkey)
    # Build request
    headers = {'Content-Type': 'application/x-www-form-urlencoded'}
    payload = {'appid': appid, 'q': query, 'from': from_lang, 'to': to_lang, 'salt': salt, 'sign': sign}
    # Send request
    r = requests.post(url, params=payload, headers=headers)
    result = r.json()
    return result['trans_result'][0]['dst']

def write2txt(path,title,contents):
    with open(path,"w") as f:
        for ti,con in zip(title,contents):
            f.write(ti+"\n")
            f.write(con+"\n")
            f.write("\n")

In [88]:
import requests
from bs4 import BeautifulSoup
import re
import random
from hashlib import md5
import time

In [89]:
web = "https://openaccess.thecvf.com/ICCV2021?day=2021-10-12"
name,pdfurls = getUrl(web)
abstracts = getAbstract(pdfurls)
ch_abstracts = []


In [ ]:
for ab in abstracts:
    temp = translate_api(ab)
    ch_abstracts.append(temp)
    time.sleep(1)
write2txt("./my_abstract.txt",name,ch_abstracts)